# **Image Denoising**

<br>

* Image Denoising is the process of removing noise from the Images

* The noise present in the images may be caused by various intrinsic or extrinsic conditions which are practically hard to deal with. 
* The problem of Image Denoising is a very fundamental challenge in the domain of Image processing and Computer vision. 
* Therefore, it plays an important role in a wide variety of domains where getting the original image is really important for robust performance.
<br>

<img src="https://dmitryulyanov.github.io/assets/deep-image-prior/teaser/cropped_02.png">

* Traditional image denoising algorithms always assume the noise to be homogeneous Gaussian distributed. However, in practice, the noise on real images can be much more complex. Such noise on real images is called Real-noise or Blind-noise. Traditional filters fail to perform well on images with such noise.

* We can remove the noise from the noisy images using **autoencoders** or **encoder-decoder networks**

##**Autoencoders**

* Autoencoder is an unsupervised artificial neural network that is trained to copy its input to output. 
* In the case of image data, the autoencoder will first encode the image into a lower-dimensional representation, then decodes that representation back to the image. 
* Encoder-Decoder automatically consists of the following two structures:

  1. The encoder- This network downsamples the data into lower dimensions.
  2. The decoder- This network reconstructs the original data from the lower dimension representation.
* The lower dimension (i.e, output of encoder network) representation is usually known as latent space representation.
<br>
<img src="https://miro.medium.com/max/750/1*sahNK4wy4teFA0r6tJJwKQ.png">

##**How Autoencoders Work ?**

* The network is provided with original images x, as well as their noisy version x~. The network tries to reconstruct its output x’ to be as close as possible to the original image x. By doing so, it learns how to denoise images.

* the encoder model turns the input into a small dense representation. The decoder model can be seen as a generative model which is able to generate specific features.

* Both encoder and decoder networks are usually trained as a whole. The loss function penalizes the network for creating output x’ that differs from the original input x.
* By doing so the encoder learns to preserve as much of the relevant information needed in the limitation of the latent space, and cleverly discard irrelevant parts, e.g. noise. The decoder learns to take the compressed latent information and reconstruct it into a full error-free input.


##**Hyperparameters Of Autoencoders**

* There are mainly 4 parameters that we need to set before training the autoencoder
* **Code size**: This represents the number of nodes in the middle layer. The smaller the code size more the compression is and if you want less compression then increase the code size.
* **Number of Layers**: In the above architecture image there are only 2 layers in encoder and decoder but we can make it as deep we want it to be.
* **The number of nodes per layer**: Usually what happens is the number of nodes per layer decreases with each subsequent layer of an encoder and then starts increasing again with each subsequent layer of the decoder. The decoder is symmetric to the structure of the encoder but that’s not a requirement.
* **Loss function**: The popular choices here are mean squared error(MSE) or binary cross-entropy. If the input values are in the range [0,1] then binary cross-entropy is favored as compared to MSE. Otherwise, we just use MSE.

##**Implementaion**

* Let's implement an autoencoder to denoise hand-written digits. The input is a 28x28 grey scaled image, building a 784-elements vector.
* The encoder network is a single dense layer with 64 neurons. Therefore the latent space will have dimension 64. A rectified units (ReLu) activation function is attached to each neuron in the layer, and determines whether it should be activated (“fired”) or not, based on whether each neuron’s input is relevant for the autoencoder’s prediction. * The activation function also helps normalize the output of each neuron to a range between 1 and 0.
* The decoder network is a single dense layer with 784 neurons, corresponding to a 28x28 greyscaled output image. A sigmoid activation function is used to compare the encoder input versus the decoder output.
* Binary cross-entropy is used as a loss function and Adadelta as an optimizer for minimizing the loss function.

In [ ]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import numpy as np

# input layer
input_img = Input(shape=(784,))

# autoencoder
encoding_dim = 32  
encoded = Dense(encoding_dim, activation='relu')(input_img)
encoded_input = Input(shape=(encoding_dim,))
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input_img, decoded)
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

The [MNIST](http://yann.lecun.com/exdb/mnist/) dataset is a well-known database of handwritten digits that is widely used for training and testing in the field of machine learning. We use it here to generate synthetic noisy digits by applying a Gaussian noise matrix and clipping the images between 0 and 1.


In [ ]:
import matplotlib.pyplot as plt
import random
%matplotlib inline

# get MNIST images, clean and with noise
def get_mnist(noise_factor=0.5):
  (x_train, y_train), (x_test, y_test) = mnist.load_data()

  x_train = x_train.astype('float32') / 255.
  x_test = x_test.astype('float32') / 255.
  x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
  x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

  x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
  x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

  x_train_noisy = np.clip(x_train_noisy, 0., 1.)
  x_test_noisy = np.clip(x_test_noisy, 0., 1.)
  
  return x_train, x_test, x_train_noisy, x_test_noisy, y_train, y_test

x_train, x_test, x_train_noisy, x_test_noisy, y_train, y_test = get_mnist()

# plot n random digits
# use labels to specify which digits to plot
def plot_mnist(x, y, n=10, randomly=False, labels=[]):
  plt.figure(figsize=(20, 2))
  if len(labels)>0:
    x = x[np.isin(y, labels)]
  for i in range(1,n,1):
      ax = plt.subplot(1, n, i)
      if randomly:
        j = random.randint(0,x.shape[0])
      else:
        j = i
      plt.imshow(x[j].reshape(28, 28))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
  plt.show()
  
plot_mnist(x_test_noisy, y_test, randomly=True)

The autoencoder will minimize the difference between noisy and clean images. By doing this it will learn how to remove noise from any unseen hand-written digit, that was produced with similar noise.

In [ ]:
# flatten the 28x28 images into vectors of size 784.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
x_train_noisy = x_train_noisy.reshape((len(x_train_noisy), np.prod(x_train_noisy.shape[1:])))
x_test_noisy = x_test_noisy.reshape((len(x_test_noisy), np.prod(x_test_noisy.shape[1:])))

#training
history = autoencoder.fit(x_train_noisy, x_train,
                          epochs=100,
                          batch_size=128,
                          shuffle=True,
                          validation_data=(x_test_noisy, x_test))
                          
# plot training performance
def plot_training_loss(history):

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(1, len(loss) + 1)

  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title('Training and validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  plt.show()

plot_training_loss(history)

<img src="https://miro.medium.com/max/602/1*PLBqcw1kPpsjMcij3VO6pA.png">

Now we can use the trained autoencoder to clean unseen noisy input images and plot them against their cleaned version.

In [ ]:
# plot de-noised images
def plot_mnist_predict(x_test, x_test_noisy, autoencoder, y_test, labels=[]):
  
  if len(labels)>0:
    x_test = x_test[np.isin(y_test, labels)]
    x_test_noisy = x_test_noisy[np.isin(y_test, labels)]

  decoded_imgs = autoencoder.predict(x_test)
  n = 10  
  plt.figure(figsize=(20, 4))
  for i in range(n):
      ax = plt.subplot(2, n, i + 1)
      plt.imshow(x_test_noisy[i].reshape(28, 28))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)

      ax = plt.subplot(2, n, i + 1 + n)
      plt.imshow(decoded_imgs[i].reshape(28, 28))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
  plt.show()
  return decoded_imgs, x_test
 
decoded_imgs_test, x_test_new = plot_mnist_predict(x_test, x_test_noisy, autoencoder, y_test)

<img src="https://miro.medium.com/max/1050/1*pzPbp765q8NlgJWg2PXXRQ.png">

Overall, the noise is removed very well. The white dots which were introduced artificially on the input images have disappeared from the cleaned images. The digits can be recognized visually.

In this article, I described an image denoising technique with a practical guide on how to build autoencoders with Python.I hope you got Basic Understanding about denoising the images.

Thanks for reading !

##**References**

https://www.jeremyjordan.me/autoencoders/

https://www.tensorflow.org/tutorials/generative/autoencoder